**Import statements**


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import constants
import functions4 as functions

%matplotlib inline

**Load Data**

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

**Compute predictions for stationary sensors**

In [9]:
mae = []
mse = []
squares = []

start_time = '2018-07-03 00:00:00'
start_window = '2018-07-04 13:30:00'
end_window = '2018-07-04 13:45:00'

number_of_windows = 4
window = 15
par_grid = functions.create_par(c=0.00002, epsilon=0.001)
timeint_on_first_window = 0
tmp_to_test_filename = 'tmp_to_test/with_mobile_data_{}_{}.csv'.format(start_window, number_of_windows)

for i in range(1, number_of_windows+1):
    end_time = start_window
    # treinar sem dados moveis
    # testar em dados fixos em vez de móveis
    #print(start_window)
    #print(end_window)
    timeint, par_grid = functions.train(all_static_data, mobile_sensor_data, start_time, end_time, par_grid, window, timeint_on_first_window)
    to_test = functions.test_mobile(par_grid, timeint, mobile_sensor_data, start_window, end_window, 1)
    #print(to_test)
    
    # Calcular o erro numa das colunas do to_test
    to_test['error_PM2.5'] = to_test['PM2.5'] - to_test['pred_PM2.5']
    to_test['start_window'] = start_window

    # Meter num csv
    to_test[['start_window', 'lat_grid', 'long_grid', 'PM2.5', 'pred_PM2.5', 'error_PM2.5']].to_csv(tmp_to_test_filename, mode='a', index=False)
    # e depois deixar uma linha branca para separar.
    with open(tmp_to_test_filename, 'a') as f:
        f.write("\n")
    
    squares.append(to_test.shape[0])
    mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
    mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))

    start_time = end_time
    start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
    end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
    timeint_on_first_window = timeint+1
    

In [6]:
print(mae)
print(squares)

[0.09606279052747668, 2.497366581097405, 2.094269625578577, 3.0606891640250438]
[1, 25, 29, 23]


In [23]:
mae = []
mse = []
squares = []

start_time = '2018-07-03 00:00:00'
start_window = '2018-07-06 14:00:00'
end_window = '2018-07-06 14:15:00'

number_of_windows = 6
window = 15
par_grid = functions.create_par(c=0.00002, epsilon=0.001)
timeint_on_first_window = 0

for i in range(1, number_of_windows+1):
    end_time = start_window
    # treinar sem dados moveis
    # testar em dados fixos em vez de móveis
    timeint, par_grid = functions.train(all_static_data, None, start_time, end_time, par_grid, window, timeint_on_first_window)
    to_test = functions.test_mobile(par_grid, timeint, mobile_sensor_data, start_window, end_window, 1)
    squares.append(to_test.shape[0])
    mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
    mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))

    start_time = end_time
    start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
    end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
    timeint_on_first_window = timeint+1

344
4
12
345
4
11
345
4
12
345
5
10
345
5
11
345
5
12
345
6
8
345
6
9
345
7
7
345
7
8
345
8
8
345
9
2
345
9
3
345
9
4
345
9
5
345
9
6
345
9
7
345
9
8
345
10
0
345
10
1
345
10
2
345
10
7
345
10
8
345
11
0
345
12
1
345
13
2
346
9
6
346
10
6
346
10
7
346
10
8
346
11
6
346
11
8
346
12
6
346
12
9
346
13
2
346
13
6
346
13
9
346
13
10
346
14
2
346
14
3
346
14
4
346
14
6
346
14
10
346
14
11
346
14
16
346
15
4
346
15
5
346
15
6
346
15
11
346
15
14
346
15
15
346
15
16
346
16
12
346
16
13
346
16
14
347
8
6
347
9
2
347
9
3
347
9
4
347
9
5
347
9
6
347
9
7
347
10
1
347
10
2
347
10
5
347
10
6
347
10
7
347
10
8
347
10
9
347
10
10
347
11
11
347
11
12
347
11
13
347
11
14
347
12
15
347
12
16
347
13
16
347
14
16
348
2
7
348
2
8
348
2
9
348
2
10
348
3
6
348
3
7
348
3
10
348
3
11
348
3
12
348
4
6
348
4
11
348
4
12
348
4
13
348
5
6
348
5
10
348
5
11
348
5
12
348
6
6
348
6
8
348
6
9
348
6
11
348
7
6
348
7
8
348
7
11
348
8
8
348
8
9
348
8
10
348
8
11
348
8
12
349
0
11
349
1
10
349
1
11
349
1
12
349
2
10
349
2


In [24]:
print(mae)
print(squares)

[0.09606275163320832, 2.491253300356053, 3.0219637882844954, 3.7368385521998952, 2.2859129836854444, 1.7715089760134815]
[1, 25, 29, 23, 29, 11]
